In [1]:
import cobra
import pandas as pd
import re
import numpy as np
import scipy.stats as st
from matplotlib import pyplot as plt
from pathlib import Path
import sys
sys.path.append('../../code/')
import leakage, utils
import pubchempy as pcp
import seaborn as sns
import time

# Settings / choices

In [2]:
only_significant_changes = False
timepoints = np.arange(1.5, 13, 1)
knock_outs = False
shadow_price_for_leaked_mets = False
species = 'yeast'

In [3]:
def inf_to_100000(model):
    for r in model.reactions:
        if np.isinf(r.lower_bound):
            r.lower_bound = -100000
        if np.isinf(r.upper_bound):
            r.upper_bound = 100000
    return model

def print_summary(model, solution):
    # Print uptake
    for r in model.exchanges:
        flux = solution[r.id]
        if  flux >1e-3:
            print(r.id, r.name,flux, r.bounds)
        elif flux < -1e-3:
            print(r.id, r.name,flux, r.bounds)
    #print("## Demands ##")
    #for r in model.demands:
    #    print(r.id, solution[r.id])

    print("Growth: ", solution['r_2111'])
    print("Prot pool exchange: ", solution['prot_pool_exchange'])
    
def get_exchange_met(model, mc_id):
    mc = model.metabolites.get_by_id(mc_id)
    me_name = mc.name.replace('[cytoplasm]', '[extracellular]')
    return [m for m in model.metabolites if m.name == me_name]

In [4]:
fn = '../../models/yeast/ecYeastGEM_batch_8_3_4_no_rev_ex.xml'
model = cobra.io.read_sbml_model(fn)
model.solver = 'gurobi'
model = inf_to_100000(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26


In [5]:
def get_leakage(time):
    exometabolites_folder = Path("../../data/{0}/".format(species))
    leakage_df = leakage.get_leakage(exometabolites_folder, species, time = time, unit = '/gDW', method = 'one-way-diff',
                                    only_significant_changes = only_significant_changes)
    leakage_df.set_index("Metabolite", inplace=True)
    leakage_df.drop_duplicates(inplace=True)
    leakage_label = "Leakage (mmol/gDW/h)"
    return leakage_df

In [6]:
print_summary(model, model.optimize())

r_1672 carbon dioxide exchange 32.98977124357251 (0.0, 100000)
r_1761 ethanol exchange 29.566489785990203 (0.0, 100000)
r_1793 formate exchange 0.016150224788539704 (0.0, 100000)
r_2100 water exchange 13.76574682590952 (0.0, 100000)
r_4527 diphosphate exchange 3.547990276340295 (0.0, 100000)
r_1654_REV ammonium exchange (reversible) 2.5816166708064148 (0.0, 100000)
r_1714_REV D-glucose exchange (reversible) 17.93092040604542 (0.0, 1000.0)
r_1832_REV H+ exchange (reversible) 20.4926800685532 (0.0, 100000)
r_1992_REV oxygen exchange (reversible) 2.701685147415501 (0.0, 100000)
r_2005_REV phosphate exchange (reversible) 7.199026976953928 (0.0, 100000)
r_2020_REV potassium exchange (reversible) 0.001367874213516301 (0.0, 100000)
r_2049_REV sodium exchange (reversible) 0.001495994663267139 (0.0, 100000)
r_2060_REV sulphate exchange (reversible) 0.0344959412231053 (0.0, 100000)
r_4595_REV Mn(2+) exchange (reversible) 0.0010287318465287882 (0.0, 100000)
Growth:  0.3768248522083474
Prot pool e

In [7]:
# model.reactions.get_by_id('r_1761_REV').upper_bound = 1
# model.reactions.get_by_id('r_1714_REV').upper_bound = 0
# print_summary(model, model.optimize())

In [8]:
exometabolites_folder = Path("../../data/{0}/".format(species))


In [9]:
met_info_df = pd.read_csv("../../data/met_info_curated.csv", encoding = "ISO-8859-1", index_col = 0)

In [10]:
# Read metabolite mapping
mapping_df = pd.read_csv('../../data/id_mapping.csv', index_col=0)
# df2 = pd.merge(leakage_df, mapping_df, left_index=True, right_index=True)
# df2.drop(columns='Metabolite name', inplace=True)

# Get leakage


In [11]:
glucose_exchange_id = 'r_1714_REV'
split_reactions = True

# timepoints = np.arange(1.5,8, 1)#[5,6,7,8,9,10,11, 12, 13]
for i, t in enumerate(timepoints):
    print(t)
    leakage_df = get_leakage(t)
    # Consider to use an earlier time-point
    glucose_uptake_rate = leakage.get_glucose_uptake_rate(exometabolites_folder, "yeast", time = t, method = 'one-way-diff')
    df2 = pd.merge(leakage_df, mapping_df, left_index=True, right_index=True)
    df2.drop(columns='Metabolite name', inplace=True)
    df = pd.merge(met_info_df, df2, left_on = 'Metabolite id', right_on = 'Ecoli metabolite')
    df['Time'] = t
    df['Glucose'] = -glucose_uptake_rate
    print(glucose_uptake_rate)
    # Set model constraints
    with model:
        if split_reactions:
            model.reactions.get_by_id(glucose_exchange_id).upper_bound = max(-1*glucose_uptake_rate, 0)
        else:
            model.reactions.get_by_id(glucose_exchange_id).lower_bound = min(glucose_uptake_rate, 0)
        for j, row in df.iterrows():
            if row['Leakage (mmol/gDW/h)'] < 0:
                met_ids = row['Yeast metabolite'].split(',')
                mets = []
                leak_rate =  row['Leakage (mmol/gDW/h)'] 
                for m_id in met_ids:
                    mc_id = "{0}[c]".format(m_id.strip())
                    try:
                        m = get_exchange_met(model, mc_id)[0]
                    except (KeyError, IndexError) as e:
                        mc = model.metabolites.get_by_id(mc_id)
                        print("Could not find {0}: {1}".format(mc_id, mc.name))
                        continue
                    else:
                        mets.append(m)
                for m in mets:
                    r_exs = [r for r in m.reactions if len(r.metabolites)==1]
                    if split_reactions:
                        r_ex = [r for r in r_exs if 'REV' in r.id][0]
                        r_ex.upper_bound = -leak_rate/len(mets)
                    else:
                        r_ex = r_exs[0]
                        r_ex.lower_bound = leak_rate/len(mets)
                    # Should check soplutions
                    # print(r_ex.id)
                    # print("Setting uptake rate of {0} to {1} with reaction {2}, {3}".format(m.name, 
                                                    # -leak_rate, r_ex, r_ex.id))
        solution = model.optimize()
        print(solution.objective_value)
        pfba_solution = cobra.flux_analysis.pfba(model)
        # List already excreted metabolites
        exchanged_mets = {}
        for r in model.boundary:
            flux = solution.fluxes[r.id]
            if flux != 0:
                exchanged_mets[list(r.metabolites.keys())[0].id[:-2]]=(r.id, flux)
        
        # print(model.summary())
        df['Predicted growth rate'] = solution.objective_value
        # Get turnover and shadow prices
        turnover = {}
        shadow_prices = {}        
        for j, row in df.iterrows():
            if row['Leakage (mmol/gDW/h)'] > 0:
                met_ids = row['Yeast metabolite'].split(',')
                sp_list = []
                turnover_list = []
                for key in met_ids:
                    if key.strip() in exchanged_mets.keys():
                        if shadow_price_for_leaked_mets:
                            # the change should be taken into account the already exchanged flux
                            existing_flux = exchanged_mets[key.strip()]
                        else:
                            continue
                    else:
                        existing_flux = None
                    m_id = "{0}[c]".format(key.strip())
                    m = model.metabolites.get_by_id(m_id)
                    sp_list.append(leakage.estimate_shadow_price_for_met(model, m, solution, delta = 0.01, existing_flux = existing_flux))
                    # turnover_list.append(get_turnover_flux(m, pfba_solution))
                    turnover_list.append(m.summary(pfba_solution).producing_flux['flux'].sum())
                # print(met_ids, sp_list, turnover_list)
                # Shadow prices
                if len(sp_list):
                    shadow_prices[j] = np.nanmean(sp_list)
                    turnover[j] = np.mean(turnover_list)
                else:
                    shadow_prices[j] = np.nan
                    turnover[j] = np.nan
                # print(met_ids, np.nanmean(sp_list))
        df["Shadow price"] = pd.Series(shadow_prices)
        df["Turnover"] = pd.Series(turnover)
        
    if i == 0:
        full_df = df
    else:
        full_df = pd.concat([full_df, df])


1.5
-10.763953320088106
0.36702610729307134
2.5
-4.817361276437225
0.3287940895795481
3.5
-13.167743226178732
Could not find s_0555[c]: D-fructose 1,6-bisphosphate [cytoplasm]
0.37386957150349964
4.5
-13.149881488903473
Could not find s_1269[c]: orotate [cytoplasm]
0.3724445264737478
5.5
-1.713688982582195
0.15201144258748223
6.5
-1.3314044955834639
Could not find s_1269[c]: orotate [cytoplasm]
0.11956485740289446
7.5
0.0
Could not find s_0555[c]: D-fructose 1,6-bisphosphate [cytoplasm]
Could not find s_0629[c]: dihydroxyacetone phosphate [cytoplasm]
Could not find s_0577[c]: D-ribulose 5-phosphate [cytoplasm]
Could not find s_0581[c]: D-xylulose 5-phosphate [cytoplasm]
Could not find s_0516[c]: cis-aconitate [cytoplasm]
0.07401285750332104
8.5
0.0
Could not find s_0557[c]: D-fructose 6-phosphate [cytoplasm]
0.007137602233672872
9.5
0.0
Could not find s_0555[c]: D-fructose 1,6-bisphosphate [cytoplasm]
0.0913752831268162
10.5
0.0
Could not find s_0629[c]: dihydroxyacetone phosphate [cyt

In [12]:
# full_leakage['Uptake (mmol/gDW/h)'] = 0
new_df = full_df.copy()

In [13]:
new_df['Uptake (mmol/gDW/h)'] = -1*new_df['Leakage (mmol/gDW/h)']

In [14]:
new_df.loc[new_df['Leakage (mmol/gDW/h)'] < 0, 'Leakage (mmol/gDW/h)'] = np.nan
new_df.loc[new_df['Uptake (mmol/gDW/h)'] < 0, 'Uptake (mmol/gDW/h)'] = 0


In [15]:
new_df['log10(Leakage [mmol/gDW/h])'] = np.log10(new_df['Leakage (mmol/gDW/h)'])
new_df['log10(-Shadow price [gDW/mmol)'] = np.log10(-new_df['Shadow price'])
new_df['log10(Turnover [mmol/gDW/h])'] = np.log10(new_df['Turnover']).replace(-np.inf, np.nan)

/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [16]:
new_df.drop(columns=['Value', 'Uncertainty', 'Ecoli metabolite'], inplace = True)#'Yeast metabolite',

In [17]:
timestr = time.strftime("%Y%m%d")

if only_significant_changes:
    s1 = '_osc'
else:
    s1 = ''

if shadow_price_for_leaked_mets:
    s2 = '_SP_for_leaked'
else:
    s2 = ''

if knock_outs:
    s3 = '_KO'
else:
    s3 = ''
fn = 'spreadsheet_{0}_leakage_{1}{2}{3}{4}.csv'.format(species, timestr, s1, s2, s3)
folder = Path('../../results/{0}/'.format(species))
new_df.to_csv(folder / fn)

In [18]:
full_leakage = full_df.loc[~full_df['Shadow price'].isna(), :]
full_leakage = full_leakage.loc[full_leakage.Turnover <100, :]
full_leakage = full_leakage.loc[full_leakage['Leakage (mmol/gDW/h)'] > 0, :]

In [19]:
full_leakage['log10(leakage)'] = np.log10(full_leakage['Leakage (mmol/gDW/h)'])
full_leakage['log10(-Shadow price)'] = np.log10(-full_leakage['Shadow price'])
full_leakage['log10(Turnover)'] = np.log10(full_leakage['Turnover'])

/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
